In [3]:
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures

In [4]:
import os
all_txt_files =[]
# get all file names for obits in txt folder
for root, dirs, files in os.walk("txt"):
    for file in files:
        if file.endswith(".txt"):
            all_txt_files.append(os.path.join(root, file))
n_files = len(all_txt_files)
all_txt_files[365]

'../tf-idf-programming-historian/txt/0426.txt'

In [15]:
# create a list of strings. Each string is an obit
all_text = []
for i in all_txt_files:
    with open(i) as f:
        txt = f.read()
    all_text.append(txt)

In [51]:
all_text[0][:100]

'Late in the afternoon of Friday, Feb. 24, 1956, a short, rotund, round-headed, gleamingly bald, bagg'

In [93]:
import spacy
nlp = spacy.load('en')

# define a function to tokenize, lowercase all, remove punctuation, remove numbers etc.
def text_cleanup(mystring):
    x = ['SPACE', 'PUNCT', 'SYM', 'X', 'NUM']
    doc = nlp(mystring.lower())
    processed = [i.text for i in doc if i.pos_ not in x]
    return processed

In [94]:
# this code will take a long time to execute ... 
all_docs = [text_cleanup(i) for i in all_text

In [95]:
all_docs_merged = []
for i in all_docs:
    # insert seven instances of the string '$$$$' between each obit so collocations don't cross documents 
    all_docs_merged.extend(i)
    all_docs_merged.extend(['$$$$','$$$$','$$$$','$$$$','$$$$','$$$$','$$$$'])
    
all_docs_merged[6537:6550]

['authentic',
 'material',
 '$$$$',
 '$$$$',
 '$$$$',
 '$$$$',
 '$$$$',
 '$$$$',
 '$$$$',
 'a',
 'man',
 'of',
 'missionary']

In [96]:
type(all_docs_merged[0])

str

In [97]:
#instantiate BigramAssocMeasures()
bigram_measures = BigramAssocMeasures()

# initialize the bigram collocation finder object to find and rank collocations
finder = BigramCollocationFinder.from_words(all_docs_merged, window_size=7)

# consider only collocations for terms that appear at least three times in the corpus
finder.apply_freq_filter(2)

# list of all possible measures: .raw_freq, .pmi, .likelihood_ratio, .chi_sq, .phi_sq, .fisher, .student_t, .mi_like, .poisson_stirling, .jaccard, .dice
results = finder.nbest(bigram_measures.pmi, 10)

In [106]:
scores = finder.score_ngrams(bigram_measures.pmi)

#check length of scores
len(scores)

15121

In [107]:
# this bit of code loops through the scores list and saves only results that include the seed word
# in this case, the seed word is 'life' (seems like a good seed for obits)
seed = "life"
result = []
for terms, score in scores:
    if terms[0] == seed:
        result.append([terms[0], terms[1], score])
    if terms[1] == seed:   
        result.append([terms[0], terms[1], score])

In [110]:
# result is a list of lists; each list is a row of data
result[:10]

[['life', 'cycle', 5.797787832454242],
 ['life', 'span', 5.797787832454242],
 ['mold', 'life', 5.797787832454242],
 ['life', 'experiences', 5.212825331733087],
 ['middle', 'life', 4.4758597375668785],
 ['purpose', 'life', 4.4758597375668785],
 ['writing', 'life', 4.4758597375668785],
 ['stage', 'life', 4.3827503331754],
 ['&', 'life', 3.9904329103966383],
 ['fame', 'life', 3.9904329103966383]]

In [111]:
# I can now save a csv of my collocations for the seed word
import pandas as pd
df = pd.DataFrame.from_records(result, columns=["word1", "word2", "score"])
df.to_csv("collocations_life.csv")